# COVID-19 Mortality

The objective of this Notebook is to count COVID-19 mortalities in Philippine provinces by age group. The COVID-19 data was downloaded from the Department of Health COVID-19 Tracker (https://doh.gov.ph/covid19tracker) on June 6, 2023.

Output: "output/mortality.xlsx"

In [1]:
import pandas as pd
# import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/SMSL/COVID-19 Mortality Rates

Mounted at /content/drive
/content/drive/My Drive/SMSL/COVID-19 Mortality Rates


## Exploratory Data Analysis

- What are the provinces in the province attribute of the dataset?
- How many rows do not contain a province attribute?
- How many rows do not contain a health status (died/recovered) attribute?
- How many rows do not contain an age group attribute?

In [3]:
# Read CSV files
# df = pd.DataFrame()
# for i in range(5):
#     temp = pd.read_csv(f"data/Copy of DOH COVID Data Drop_ 20230527 - 04 Case Information_batch_{i}.csv")
#     df = pd.concat([df, temp])
#     print(f"File {i} shape: {temp.shape}")

df = pd.read_csv("data/combined_covid_case_information_20230527.csv")
n, _ = df.shape
print(f"DF shape: {n}")

<ipython-input-3-d8df34078d92>:8: DtypeWarning: Columns (7,8,10,15,16,18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/combined_covid_case_information_20230527.csv")


DF shape: 4137440


In [4]:
# Columns of df
df.columns

Index(['CaseCode', 'Age', 'AgeGroup', 'Sex', 'DateSpecimen',
       'DateResultRelease', 'DateRepConf', 'DateDied', 'DateRecover',
       'RemovalType', 'Admitted', 'RegionRes', 'ProvRes', 'CityMunRes',
       'CityMuniPSGC', 'BarangayRes', 'BarangayPSGC', 'HealthStatus',
       'Quarantined', 'DateOnset', 'Pregnanttab', 'ValidationStatus'],
      dtype='object')

In [5]:
# Convert Dates to DateType
for column in ['DateSpecimen', 'DateResultRelease', 'DateRepConf', 'DateDied', 'DateRecover', 'DateOnset']:
    df[column] = pd.to_datetime(df[column])

In [6]:
# Null Values: %
df.isna().sum() / n * 100

CaseCode              0.000024
Age                   0.287617
AgeGroup              0.287617
Sex                   0.000024
DateSpecimen         25.116618
DateResultRelease    25.131506
DateRepConf           0.000000
DateDied             98.409669
DateRecover          81.980814
RemovalType           0.391450
Admitted              2.655579
RegionRes             0.093754
ProvRes               1.391827
CityMunRes            2.422754
CityMuniPSGC          2.567070
BarangayRes           8.950390
BarangayPSGC          9.091564
HealthStatus          0.000000
Quarantined           1.201709
DateOnset            64.601348
Pregnanttab          49.129607
ValidationStatus      5.352150
dtype: float64

In [7]:
# Unique Age Groups
df['AgeGroup'].unique()

array(['35 to 39', '40 to 44', '60 to 64', '45 to 49', '55 to 59',
       '30 to 34', '80+', '70 to 74', '50 to 54', '25 to 29', '65 to 69',
       '75 to 79', '20 to 24', '10 to 14', '15 to 19', '5 to 9', '0 to 4',
       nan], dtype=object)

In [8]:
# Unique Provinces
df['ProvRes'].unique()

array(['NEGROS ORIENTAL', 'BOHOL', 'BATANGAS', 'RIZAL', nan, 'NCR',
       'BULACAN', 'CAVITE', 'CAMARINES SUR', 'TARLAC',
       'SURIGAO DEL NORTE', 'BATAAN', 'PAMPANGA', 'LAGUNA', 'CEBU',
       'LANAO DEL SUR', 'PALAWAN', 'LA UNION', 'CAGAYAN',
       'NEGROS OCCIDENTAL', 'DAVAO DEL SUR', 'ILOILO', 'NUEVA ECIJA',
       'LANAO DEL NORTE', 'QUEZON', 'MARINDUQUE',
       'COTABATO CITY (NOT A PROVINCE)', 'ZAMBOANGA DEL SUR',
       'MISAMIS ORIENTAL', 'SULTAN KUDARAT', 'PANGASINAN', 'ZAMBALES',
       'BENGUET', 'NUEVA VIZCAYA', 'ALBAY', 'ISABELA', 'AKLAN',
       'OCCIDENTAL MINDORO', 'ORIENTAL MINDORO', 'AGUSAN DEL NORTE',
       'ROMBLON', 'LEYTE', 'DAVAO ORIENTAL', 'DAVAO DEL NORTE',
       'ILOCOS SUR', 'SOUTH COTABATO', 'ABRA', 'ILOCOS NORTE', 'CAPIZ',
       'SAMAR (WESTERN SAMAR)', 'MISAMIS OCCIDENTAL',
       'COTABATO (NORTH COTABATO)', 'MAGUINDANAO', 'CAMIGUIN',
       'DAVAO DE ORO', 'ANTIQUE', 'BUKIDNON', 'SURIGAO DEL SUR', 'IFUGAO',
       'DAVAO OCCIDENTAL', 'CAMARINES

In [9]:
# Unique Health Status
df['HealthStatus'].unique()

array(['RECOVERED', 'DIED', 'SEVERE', 'CRITICAL', 'MODERATE',
       'ASYMPTOMATIC', 'MILD'], dtype=object)

In [10]:
# New Attribute: Date of Infection Proxy
# First attribute with a value is followed: DateOnset, DateSpecimen, DateResultRelease, DateRepConf
df['DateInfectionProxy'] = df['DateOnset']

for column in ['DateSpecimen', 'DateResultRelease', 'DateRepConf']:
    replacement = df[df['DateInfectionProxy'].isna()][column]
    df.loc[replacement.index, 'DateInfectionProxy'] = replacement

In [11]:
# Max and Min Date of Infection Proxy
min = df['DateInfectionProxy'].min()
max = df['DateInfectionProxy'].max()
min, max

(Timestamp('2020-01-18 00:00:00'), Timestamp('2023-05-25 00:00:00'))

## Mortality

Study Period: January 18, 2020 - January 18, 2023

In [12]:
# Date Filtering
df = df[df['DateInfectionProxy'] >= pd.to_datetime('01-18-2020')] # >= January 18, 2020
df = df[df['DateInfectionProxy'] <= pd.to_datetime('01-18-2023')]  # <= January 18, 2023

min = df['DateInfectionProxy'].min()
max = df['DateInfectionProxy'].max()
min, max

(Timestamp('2020-01-18 00:00:00'), Timestamp('2023-01-18 00:00:00'))

In [13]:
# Filtering important columns
df = df[['AgeGroup', 'ProvRes', 'HealthStatus']]
n, _ = df.shape
n, df.columns

(4070840, Index(['AgeGroup', 'ProvRes', 'HealthStatus'], dtype='object'))

In [14]:
# Removing Rows with Null
for col in df.columns:
    df = df[df[col].notna()]
n, _ = df.shape
n

4003318

In [15]:
# Provinces
provinces = df['ProvRes'].unique()
provinces.sort()
provinces

array(['ABRA', 'AGUSAN DEL NORTE', 'AGUSAN DEL SUR', 'AKLAN', 'ALBAY',
       'ANTIQUE', 'APAYAO', 'AURORA', 'BASILAN', 'BATAAN', 'BATANES',
       'BATANGAS', 'BENGUET', 'BILIRAN', 'BOHOL', 'BUKIDNON', 'BULACAN',
       'CAGAYAN', 'CAMARINES NORTE', 'CAMARINES SUR', 'CAMIGUIN', 'CAPIZ',
       'CATANDUANES', 'CAVITE', 'CEBU',
       'CITY OF ISABELA (NOT A PROVINCE)', 'COTABATO (NORTH COTABATO)',
       'COTABATO CITY (NOT A PROVINCE)', 'DAVAO DE ORO',
       'DAVAO DEL NORTE', 'DAVAO DEL SUR', 'DAVAO OCCIDENTAL',
       'DAVAO ORIENTAL', 'DINAGAT ISLANDS', 'EASTERN SAMAR', 'GUIMARAS',
       'IFUGAO', 'ILOCOS NORTE', 'ILOCOS SUR', 'ILOILO', 'ISABELA',
       'KALINGA', 'LA UNION', 'LAGUNA', 'LANAO DEL NORTE',
       'LANAO DEL SUR', 'LEYTE', 'MAGUINDANAO', 'MARINDUQUE', 'MASBATE',
       'MISAMIS OCCIDENTAL', 'MISAMIS ORIENTAL', 'MOUNTAIN PROVINCE',
       'NCR', 'NEGROS OCCIDENTAL', 'NEGROS ORIENTAL', 'NORTHERN SAMAR',
       'NUEVA ECIJA', 'NUEVA VIZCAYA', 'OCCIDENTAL MINDORO',
   

In [16]:
# Health Status
health_status = df['HealthStatus'].unique()
health_status

array(['RECOVERED', 'DIED', 'SEVERE', 'CRITICAL', 'MODERATE'],
      dtype=object)

In [17]:
# Age Group
age_groups = df['AgeGroup'].unique()
age_groups.sort()
age_groups

array(['0 to 4', '10 to 14', '15 to 19', '20 to 24', '25 to 29',
       '30 to 34', '35 to 39', '40 to 44', '45 to 49', '5 to 9',
       '50 to 54', '55 to 59', '60 to 64', '65 to 69', '70 to 74',
       '75 to 79', '80+'], dtype=object)

In [18]:
age_groups = [
    '0 to 4',
    '5 to 9',
    '10 to 14',
    '15 to 19',
    '20 to 24',
    '25 to 29',
    '30 to 34',
    '35 to 39',
    '40 to 44',
    '45 to 49',
    '50 to 54',
    '55 to 59',
    '60 to 64',
    '65 to 69',
    '70 to 74',
    '75 to 79',
    '80+'
]

In [21]:
output = {}
for province in provinces:
    temp_df = df[df['ProvRes']==province]
    temp_output = pd.DataFrame(columns=health_status, index=age_groups)
    for status in health_status:
        temp_output[status] = temp_df[temp_df['HealthStatus']==status]['AgeGroup'].value_counts()

    # Filtering CITY OF ISABLE AND COTABATO CITY and removing (NOT A PROVINCE)
    if "(NOT A PROVINCE)" in province:
        province = province.replace("(NOT A PROVINCE)", "").strip()
    output[province] = temp_output.fillna(0)
output

{'ABRA':           RECOVERED  DIED  SEVERE  CRITICAL  MODERATE
 0 to 4          223   3.0     0.0       0.0       0.0
 5 to 9          239   0.0     0.0       0.0       0.0
 10 to 14        341   0.0     0.0       0.0       0.0
 15 to 19        472   0.0     0.0       0.0       0.0
 20 to 24        591   0.0     0.0       0.0       0.0
 25 to 29        712   8.0     0.0       0.0       0.0
 30 to 34        679   4.0     0.0       0.0       0.0
 35 to 39        536   4.0     0.0       0.0       0.0
 40 to 44        440   3.0     0.0       0.0       0.0
 45 to 49        394   8.0     0.0       0.0       0.0
 50 to 54        373   8.0     0.0       0.0       0.0
 55 to 59        362  17.0     0.0       0.0       0.0
 60 to 64        282  12.0     0.0       0.0       0.0
 65 to 69        225  23.0     0.0       0.0       0.0
 70 to 74        172  15.0     0.0       0.0       0.0
 75 to 79         78  12.0     0.0       0.0       0.0
 80+             167  39.0     0.0       0.0       0.0,
 

In [22]:
with pd.ExcelWriter("output/mortality.xlsx") as writer:
    for province in output:
        output[province].to_excel(writer, sheet_name=province[:31]) # Excel sheets limit the sheet name length to 31
        print(f"Province {province} added.")
print("File saving completed.")

Province ABRA added.
Province AGUSAN DEL NORTE added.
Province AGUSAN DEL SUR added.
Province AKLAN added.
Province ALBAY added.
Province ANTIQUE added.
Province APAYAO added.
Province AURORA added.
Province BASILAN added.
Province BATAAN added.
Province BATANES added.
Province BATANGAS added.
Province BENGUET added.
Province BILIRAN added.
Province BOHOL added.
Province BUKIDNON added.
Province BULACAN added.
Province CAGAYAN added.
Province CAMARINES NORTE added.
Province CAMARINES SUR added.
Province CAMIGUIN added.
Province CAPIZ added.
Province CATANDUANES added.
Province CAVITE added.
Province CEBU added.
Province CITY OF ISABELA added.
Province COTABATO (NORTH COTABATO) added.
Province COTABATO CITY added.
Province DAVAO DE ORO added.
Province DAVAO DEL NORTE added.
Province DAVAO DEL SUR added.
Province DAVAO OCCIDENTAL added.
Province DAVAO ORIENTAL added.
Province DINAGAT ISLANDS added.
Province EASTERN SAMAR added.
Province GUIMARAS added.
Province IFUGAO added.
Province ILO